In [1]:
import os
import sys
import random
import math
from copy import copy, deepcopy
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import cv2
import scipy
import pandas as pd
import pixiedust
from pathlib import Path
# load numpy array from npy file
from numpy import load
from scipy.stats import multivariate_normal
from scipy.optimize import linear_sum_assignment
import heapq
# save numpy array as npy file
from numpy import asarray
from numpy import save

Pixiedust database opened successfully


In [15]:
# HERE IS WHERE I CHANGE VIDEO LOCATION AND TRACKER TYPE FOR SAVING AND LOADING DATA:
# Specific video location
Video = "Short_Garden"
# Tracking type
Tracker = "_hung"
# NOW EVERYTHING ELSE GETS DONE AUTOMATICALLY

# Root directory of the project
ROOT_DIR = os.path.abspath("C:/Users/ddefr/OneDrive/Documents/Skripsie/Mask_RCNN")

# Directory for all videos
all_vids = Path("C:/Users/ddefr/OneDrive/Documents/skripsie_videos")

#Specific video dir
video_dir = all_vids / Video

# Measurements to load:
load_measurement = video_dir / "measurements.npy"

# Result save locations:
mu_save = os.path.join(video_dir,'mu{}.npy'.format(Tracker))
cov_save = os.path.join(video_dir,'cov{}.npy'.format(Tracker))
mu_obs_save = os.path.join(video_dir,'mu_obs{}.npy'.format(Tracker))
cov_obs_save = os.path.join(video_dir,'cov_obs{}.npy'.format(Tracker))
number_of_detections_save = os.path.join(video_dir,'number_of_detections{}.npy'.format(Tracker))

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library

from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco
%matplotlib inline

In [16]:
# load array
measurement = load(load_measurement, allow_pickle=True)

In [17]:
#Kalman Gain:
def Kalman_Gain(X_cov_0, Y_transition, Ny_cov):
    if np.all(X_cov_0 != None):
        Kg = X_cov_0.dot(Y_transition.T).dot(np.linalg.inv(np.dot(Y_transition,X_cov_0).dot(Y_transition.T) + Ny_cov))
    else:
        Kg = None
    return(Kg)

#joint distribution:
def joint_dis(cov1, mu1, Y_transition, Ny_cov):
    if np.all(cov1 != None) and np.all(mu1 != None):
        cov_new = np.array([[cov1, cov1.dot(Y_transition.T)],[Y_transition.dot(cov1.T), Y_transition.dot(cov1).dot(Y_transition.T) + Ny_cov]])
        mu_new = np.array([[mu1],[Y_transition.dot(mu1)]])
    else:
        cov_new = None
        mu_new = None
    return(cov_new,mu_new)

#observe evidence
def observe(X_cov_0, X_mu_0, Y_transition, Ny_cov, y, Kg):
    if np.all(X_cov_0 != None) and np.all(X_mu_0 != None):
        X_cov_obs = X_cov_0 - Kg.dot(Y_transition).dot(X_cov_0)
        X_mu_obs = X_mu_0 + Kg.dot(y - (Y_transition.dot(X_mu_0)))
    else:
        X_cov_obs = None
        X_mu_obs = None
    return(X_cov_obs, X_mu_obs)

#Prediction
def prediction(cov_obs, mu_obs, Dyn_T, Nx):
    if np.all(cov_obs != None) and np.all(mu_obs != None):
        cov_predict = Dyn_T.dot(cov_obs).dot(Dyn_T.T) + Nx
        mu_predict = Dyn_T.dot(mu_obs)
    else:
        cov_predict = None
        mu_predict = None
    return(cov_predict, mu_predict)

def prediction_2(cov_obs, mu_obs, Dyn_T, Dyn_T_2, Nx,vx,vy):
    if np.all(cov_obs != None) and np.all(mu_obs != None):
        mu_obs[2] = vx
        mu_obs[3] = vy
        cov_predict = Dyn_T_2.dot(cov_obs).dot(Dyn_T_2.T) + Nx
        mu_predict = Dyn_T.dot(mu_obs)
    else:
        cov_predict = None
        mu_predict = None
    return(cov_predict, mu_predict)


In [18]:
#Multivariate functions
def multi_gauss_pdf(mu, covariance, x, dim):
    if np.all(mu != None) and np.all(covariance != None) and np.all(x != None):
        x_m = x - mu
        ans = (1./(np.sqrt(((2*np.pi)**dim)*np.linalg.det(covariance))))*np.exp(-0.5*(np.linalg.solve(covariance,x_m).T.dot(x_m)))
        x_0 = mu - mu
        c_norm = (1./(np.sqrt(((2*np.pi)**dim)*np.linalg.det(covariance))))*np.exp(-0.5*(np.linalg.solve(covariance,x_0).T.dot(x_0)))
        ans_norm = ans/c_norm
    else:
        ans = None
        ans_norm = None
    return (ans_norm)

In [19]:
#parametrization
#Measurement info
Ny_cov = np.array([[250, 0, 0, 0],[0, 250, 0, 0],[0, 0, 100, 0],[0, 0, 0, 100]]) #Measurement noise
Y_transition = np.array([[1,0,0,0],[0,1,0,0],[0,0,0,0],[0,0,0,0]]) #Measurement Transition Matrix

#predict info
Nx = np.array([[50, 0, 0, 0],[0, 50, 0, 0],[0, 0, 25, 0],[0, 0, 0, 25]]) #Prediction noise
Dyn_T = np.array([[1,0,1,0],[0,1,0,1],[0,0,1,0],[0,0,0,1]]) #System Dynamics
Dyn_T_2 = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]]) #System Dynamics
Gx = np.array([[5, 0, 0, 0],[0, 5, 0, 0],[0, 0, 50, 0],[0, 0, 0, 50]]) #gating

In [20]:
init = 0

for n in range(len(measurement)):      
    if (init == 0)&(np.all(measurement[n]!=None)):
        #Setup Priors
        n_objects = 1500

        mu = np.zeros([len(measurement)+1, n_objects,4])
        mu_obs = np.zeros([len(measurement), n_objects,4])
        cov = np.zeros([len(measurement)+1, n_objects,4,4])
        cov_obs = np.zeros([len(measurement), n_objects,4,4])
        for m in range(n_objects):
            mu[n+1][m] = np.array([None,None,None,None]).T
            mu_obs[n][m] = np.array([None,None,None,None]).T
            cov[n+1][m] = np.array([[None,None,None,None],[None,None,None,None],[None,None,None,None],[None,None,None,None]])
            cov_obs[n][m] = np.array([[None,None,None,None],[None,None,None,None],[None,None,None,None],[None,None,None,None]])
    
            if m in range(len(measurement[0])):
                n_obj = m
                mu[n+1][n_obj] = np.array([measurement[n][m][0],measurement[n][m][1],0,0]).T
                mu_obs[n][n_obj] = np.array([measurement[n][m][0],measurement[n][m][1],0,0]).T
                cov[n+1][n_obj] = np.array([[25, 0, 0, 0],[0, 25, 0, 0],[0, 0, 5, 0],[0, 0, 0, 5]])
                cov_obs[n][n_obj] = np.array([[25, 0, 0, 0],[0, 25, 0, 0],[0, 0, 5, 0],[0, 0, 0, 5]])
        
        no_detect = np.zeros([n_objects])
        number_of_detections = np.zeros([n_objects])
        init = 1
    elif(init == 1):
        if np.any(pd.isnull(measurement[n])):
            for o in range(n_obj + 1):
                cov_obs[n][o] = cov[n][o]
                mu_obs[n][o] = mu[n][o]
                cov[n+1][o], mu[n+1][o] = prediction(cov_obs[n][o], mu_obs[n][o], Dyn_T, Nx)
                no_detect[o] = 1 + no_detect[o]
            o = 0
        else:
            a = np.zeros([n_obj + 1, len(measurement[n])+ n_obj + 1])
            cov_gate = []
            mu_gate = []
            
            for g in range (n_obj + 1):
                if np.any(pd.isnull(mu_obs[n-1][g])) or np.any(pd.isnull(cov[n][g])):   #test to see if that track still has values            
                    for q in range(len(measurement[n]) + n_obj):
                        a[g,q] = 0
                    a[g,len(measurement[n])] = 0
                else:
                    Kg_i = Kalman_Gain(cov[n][g], Y_transition, Ny_cov)
                    cov_gate = np.array([[cov[n][g][0][0], 0],[0, cov[n][g][1][1]]])
                    mu_gate = np.array([mu[n][g][0],mu[n][g][1]]).T
                    mu_3_std = np.array([mu_gate[0] - 3*np.sqrt(cov_gate[0][0]),mu_gate[1] - 3*np.sqrt(cov_gate[1][1])]).T
                    for q in range(len(measurement[n]) + n_obj +1):
                        if q < len(measurement[n]):
                            y = np.array([measurement[n][q][0],measurement[n][q][1]]).T #Measurement
                        else:
                            y = mu_3_std
                        a[g,q] = multi_gauss_pdf(mu_gate, cov_gate, y, 2)            

            # GET BEST ASSIGNMENTS:
            cost_matrix = np.ones(a.shape) - a
            assignments = linear_sum_assignment(np.array(cost_matrix))
            for o in range(len(assignments[0])):
                if assignments[1][o] < len(measurement[n]):
                    Kg = Kalman_Gain(cov[n][o], Y_transition, Ny_cov)
                    y_e = np.array([measurement[n][assignments[1][o]][0],measurement[n][assignments[1][o]][1],0,0]).T #Measurement
                    cov_obs[n][o], mu_obs[n][o] = observe(cov[n][o], mu[n][o], Y_transition, Ny_cov, y_e, Kg)
                    cov[n+1][o], mu[n+1][o] = prediction(cov_obs[n][o], mu_obs[n][o], Dyn_T, Nx)
                    no_detect[o] = 0
                    number_of_detections = number_of_detections + 1
                else:
                    if no_detect[o] > 10:
                        cov[n+1][o], mu[n+1][o] = None,None
                        mu_obs[n][o], cov_obs[n][o] =None, None
                    else:
                        cov_obs[n][o] = cov[n][o]
                        mu_obs[n][o] = mu[n][o]
                        cov[n+1][o], mu[n+1][o] = prediction(cov_obs[n][o], mu_obs[n][o], Dyn_T, Nx)
                        no_detect[o] = 1 + no_detect[o]

            #STARTING NEW TRACKS:
            m = 0
            for m in range(len(measurement[n])):
                if m not in assignments[1] and (n_obj + 1) < n_objects:
                    n_obj = n_obj + 1
                    mu[n+1][n_obj] = np.array([measurement[n][m][0],measurement[n][m][1],0,0]).T
                    mu_obs[n][n_obj] = np.array([measurement[n][m][0],measurement[n][m][1],0,0]).T
                    cov[n+1][n_obj] = np.array([[25, 0, 0, 0],[0, 25, 0, 0],[0, 0, 5, 0],[0, 0, 0, 5]])
                    cov_obs[n][n_obj] = np.array([[25, 0, 0, 0],[0, 25, 0, 0],[0, 0, 5, 0],[0, 0, 0, 5]])
                    
# save to npy file
save(mu_save, mu)
save(cov_save, cov)
save(cov_obs_save, cov_obs)
save(mu_obs_save, mu_obs)
save(number_of_detections_save, number_of_detections)